In [1]:
import numpy as np
import pdb
import xml.etree.ElementTree
import json
#from moviepy.editor import VideoFileClip
from moviepy.editor import *
import os
import glob
import xmltodict
import pandas as pd

rootdir = '/Volumes/scet/CalWWP-2_2017-18/_2017-18'
game_metadata_file = '{}/game_metadata.csv'.format(rootdir)
game_metadata = pd.read_csv(game_metadata_file)

In [69]:
game_metadata.game_file.values[25] #.game_file.values

'_CameraFiles/20180128_SJSUvCAL.mov'

In [ ]:
## get all the keys from all the xml files
all_keys = []
for (i,xml) in enumerate(game_metadata.xml_file.values):
    print(i)
    with open('{}/{}'.format(rootdir,xml)) as fd:
        temp = xmltodict.parse(fd.read())
        events = temp['file']['ALL_INSTANCES']['instance']
        for (i,e) in enumerate(events):
            all_keys.append(e['code'])

In [4]:
#len(dark_clips+white_clips)
json_filename = '/Users/panna/Documents/scet/calibration_app_grid/static/annotation/20180128_SJSUvCAL/' + str(1)+'.json'
# load existing annotation so that we just update the json, instead of overwrite
with open(json_filename,'r') as fp:
    d = json.load(fp)

In [5]:
d

{u'camera_direction': u'',
 u'events': [{u'code': u'Dark Counter Attack',
   u't_end': 111.134,
   u't_start': 86.134},
  {u'code': u'Dark Offense', u't_end': 144.384, u't_start': 104.374},
  {u'code': u'Dark Goal', u't_end': 117.864, u't_start': 104.864},
  {u'code': u'Dark Shot', u't_end': 119.874, u't_start': 106.864}],
 u'game_meta': {u'away_score': 9.0,
  u'away_team': u'San Jose State',
  u'fosh_url': nan,
  u'game_date': u'1/28/18',
  u'game_file': u'_CameraFiles/20180128_SJSUvCAL.mov',
  u'game_time': u'9:45 AM',
  u'home_score': 7.0,
  u'home_team': u'Cal',
  u'pool_location': u'Spieker Berkeley',
  u'xml_file': u'_XML_Codes/20180128_SJSUvCAL.xml'},
 u'homography': [],
 u'image_file': u'/Volumes/scet/CalWWP-2_2017-18/extracted_frames/20180128_SJSUvCAL/1.png',
 u'img_pts': [],
 u'template_pts': [],
 u'time': {u't': 112.369, u't_end': 119.874, u't_start': 104.864}}

In [8]:
meta_keys = ['game_file','xml_file','fosh_url','pool_location',
             'game_time','game_date','home_team','away_team',
             'home_score','away_score']

teams = ['Dark','White']
codes = ['{} 6 X 5','{} 6 X 5 TO','{} After Goal','{} Center Entry','{} Counter Attack',
         '{} Goal','{} Offense','{} Penalty Earned','{} Shot','{} TO']

codes_keep = ['{} Center Entry','{} Goal','{} Shot']
    
dark_clips,white_clips,meta = process_game(rootdir,25,game_metadata,meta_keys,codes,codes_keep,teams)

In [10]:
frame_save_loc = '/Volumes/scet/CalWWP-2_2017-18/extracted_clips/temp.png'
#save_frame_image(rootdir,meta,dark_clips[0],frame_save_loc)
for (i,dc) in enumerate(dark_clips[0:1]):#+white_clips):
    print(i)
    frame_save_loc = '/Volumes/scet/CalWWP-2_2017-18/extracted_frames/{}/{}.png'.format(os.path.split(meta['game_file'])[1][:-4],i)
    json_save_loc = '/Volumes/scet/CalWWP-2_2017-18/extracted_frames/{}/{}.json'.format(os.path.split(meta['game_file'])[1][:-4],i)
    save_frame_image(rootdir,meta,dc,frame_save_loc)
    save_frame_json(meta,dc,json_save_loc,frame_save_loc)


0


In [29]:
#clip_save_loc = '/Volumes/scet/CalWWP-2_2017-18/temp_clips/1.avi'
#json_save_loc = '/Volumes/scet/CalWWP-2_2017-18/temp_clips/1.json'

for (i,dc) in enumerate(dark_clips):
    clip_save_loc = '/Volumes/scet/CalWWP-2_2017-18/extracted_clips/{}/0_{}.avi'.format(os.path.split(meta['game_file'])[1][:-4],i)
    json_save_loc = '/Volumes/scet/CalWWP-2_2017-18/extracted_clips/{}/0_{}.json'.format(os.path.split(meta['game_file'])[1][:-4],i)
    #save_clip_video(rootdir,meta,dc,clip_save_loc)
    save_clip_json(meta,dc,json_save_loc,clip_save_loc)

In [55]:
clip_data = {}
clip_data['game_meta'] = meta
clip_data['time'] = {'t_start':dark_clips[0][0][0],'t_end':dark_clips[0][0][1]}
events_dict = []
for e in dark_clips[0][1:]:
    events_dict.append({'t_start':e[0],'t_end':e[1],'code':e[2]})
clip_data['events'] = events_dict
clip_data

{'events': [{'code': 'Dark Offense', 't_end': 102.59, 't_start': 62.58},
  {'code': 'Dark Center Entry', 't_end': 86.76, 't_start': 71.7},
  {'code': 'Dark Goal', 't_end': 87.74, 't_start': 74.73},
  {'code': 'Dark Shot', 't_end': 89.74, 't_start': 76.74}],
 'game_meta': {'away_score': 9.0,
  'away_team': 'San Jose State',
  'fosh_url': nan,
  'game_date': '1/28/18',
  'game_file': '_CameraFiles/20180128_SJSUvCAL.mov',
  'game_time': '9:45 AM',
  'home_score': 7.0,
  'home_team': 'Cal',
  'pool_location': 'Spieker Berkeley',
  't_start': 71.7,
  'xml_file': '_XML_Codes/20180128_SJSUvCAL.xml'},
 'time': {'t_end': 89.74, 't_start': 71.7}}

In [7]:
def get_xml_events(fname):
    with open(fname) as fd:
        doc = xmltodict.parse(fd.read())
    events = doc['file']['ALL_INSTANCES']['instance']
    #pdb.set_trace()
    return events

def get_game_metadata(game_id,game_metadata,keylist):
    metadata = game_metadata.iloc[game_id].to_dict()
    metadata = { key: metadata[key] for key in keylist }
    return metadata

def get_all_times(events,event_name):
    t = []
    for (i,e) in enumerate(events):
        if e['code'] == event_name:
            t.append((float(e['start']),float(e['end']),str(e['code'])))
    return t

def merge_ranges(ranges):
    """
    Merge overlapping and adjacent ranges and yield the merged ranges
    in order. The argument must be an iterable of triplets (start, stop, code).

    >>> input: [(37.884, 77.884, 'Dark Offense'),
                (44.684, 59.694, 'Dark Center Entry'),
                (49.264, 62.264, 'Dark Shot'),
                (86.134, 111.134, 'Dark Counter Attack'),
                (104.374, 144.384, 'Dark Offense'),
                (104.864, 117.864, 'Dark Goal'),
                (106.864, 119.874, 'Dark Shot'),
                (149.814, 174.814, 'Dark Counter Attack'),
                (157.204, 197.214, 'Dark Offense'),
                (171.454, 184.464, 'Dark Shot')]
    >>> output: [[(37.884, 77.884, 'Dark Offense'),
                  (44.684, 59.694, 'Dark Center Entry'),
                  (49.264, 62.264, 'Dark Shot')],
                 [(86.134, 111.134, 'Dark Counter Attack'),
                  (104.374, 144.384, 'Dark Offense'),
                  (104.864, 117.864, 'Dark Goal'),
                  (106.864, 119.874, 'Dark Shot')],
                 [(149.814, 174.814, 'Dark Counter Attack'),
                  (157.204, 197.214, 'Dark Offense'),
                  (171.454, 184.464, 'Dark Shot')]]
    """
    result = []
    current_start = -1
    current_stop = -1 
    current_code = ''

    for start, stop, code in sorted(ranges):
        if start > current_stop:
            # this segment starts after the last segment stops
            # just add a new segment
            result.append( [(start, stop, code)] )
            current_start, current_stop, current_code = start, stop, code
        else:
            # segments overlap, replace
            result[-1].append((start, stop, code)) 
            # current_start already guaranteed to be lower
            #current_stop = max(current_stop, stop)
    return result

def get_clip_times(event_list,codes):
    clip_times = []
    for el in event_list:
        labels = [e[2] for e in el]
        if len(set(labels).intersection(set(codes))) is not 0:
            t_start = []
            t_end = []
            for el_i in el:
                if el_i[2] in codes:
                    t_start = t_start + [el_i[0]]
                    t_end = t_end + [el_i[1]]
            clip_times = clip_times+[[(min(t_start),max(t_end))]+el]
    return clip_times

def process_game(rootdir,game_id,game_metadata,meta_keys,codes,codes_keep,teams):
    filename = '{}/{}'.format(rootdir,game_metadata.xml_file.values[game_id])
    events = get_xml_events(filename)
    meta = get_game_metadata(game_id,game_metadata,meta_keys)
    
    event_clips = []
    for team in teams:
        codes_keep_list = [ck.format(team) for ck in codes_keep]
        event_list = []
        for co in codes:
            event_list = event_list+get_all_times(events,co.format(team))
        #pdb.set_trace()
        event_list = merge_ranges(event_list)
        event_clips.append(get_clip_times(event_list,codes_keep_list))
    return event_clips[0],event_clips[1],meta

def save_clip_video(rootdir,meta,clip_info,clip_save_loc):
    # get the clip
    full_game_loc = '{}/{}'.format(rootdir,meta['game_file'])
    t_start = clip_info[0][0]
    t_end = clip_info[0][1]
    clip = VideoFileClip(full_game_loc).subclip(t_start, t_end)
    # clip name convention GAME_CODE +  event_id + clip number
    # t_end - t_start
    clip.set_duration(t_end-t_start).fl_image(invert_image).write_videofile(clip_save_loc,codec=codec)
    del clip

def save_clip_json(meta,clip_info,json_save_loc,clip_save_loc):
    # get the json
    clip_data = {}
    clip_data['game_meta'] = meta
    clip_data['time'] = {'t_start':clip_info[0][0],'t_end':clip_info[0][1]}
    clip_data['video_loc'] = clip_save_loc[31:]
    events_dict = []
    # the first element in clip info has the start and end times for the clip itself
    for e in clip_info[1:]:
        events_dict.append({'t_start':e[0],'t_end':e[1],'code':e[2]})
    clip_data['events'] = events_dict
    with open(json_save_loc, 'w') as fp:
        json.dump(clip_data, fp, sort_keys=True,indent=4, separators=(',', ': '))
        
def save_frame_image(rootdir,meta,clip_info,frame_save_loc):
    # get the clip
    full_game_loc = '{}/{}'.format(rootdir,meta['game_file'])
    t_start = clip_info[0][0]
    t_end = clip_info[0][1]
    clip = VideoFileClip(full_game_loc)
    clip.save_frame(frame_save_loc,t=(t_start+t_end)/2.)
    del clip.reader
    del clip
    # clip name convention GAME_CODE +  event_id + clip number
    # t_end - t_start
    #clip.set_duration(t_end-t_start).fl_image(invert_image).write_videofile(clip_save_loc,codec=codec)
    #del clip

def save_frame_json(meta,clip_info,json_save_loc,frame_save_loc):
    # get the json
    frame_data = {}
    frame_data['game_meta'] = meta
    # t is the time of the frame. t_start and t_end are the times of the clip from which it was pulled
    frame_data['time'] = {'t_start':clip_info[0][0],'t_end':clip_info[0][1],'t':(clip_info[0][0]+clip_info[0][1])/2.}
    frame_data['image_file'] = frame_save_loc
    events_dict = []
    # the first element in clip info has the start and end times for the clip itself
    for e in clip_info[1:]:
        events_dict.append({'t_start':e[0],'t_end':e[1],'code':e[2]})
    frame_data['events'] = events_dict
    frame_data['homography'] = []
    frame_data['img_pts'] = []
    frame_data['template_pts'] = []
    frame_data['camera_direction'] = ''
    with open(json_save_loc, 'w') as fp:
        json.dump(frame_data, fp, sort_keys=True,indent=4, separators=(',', ': '))
    
inversion = [2,1,0]; codec = 'rawvideo'
inversion = [0,1,2]; codec = 'libx264'

# for avi rawvideo, inversion: [2,1,0]
def invert_image(image):
    return image[:,:,inversion]
    

In [33]:
key = []

ids = []
for (i,e) in enumerate(events):
    key.append(e['code'])
    if e['code'] == 'Havoc':
        ids.append(i)

[[[(71.7, 89.74),
   (62.58, 102.59, 'Dark Offense'),
   (71.7, 86.76, 'Dark Center Entry'),
   (74.73, 87.74, 'Dark Goal'),
   (76.74, 89.74, 'Dark Shot')],
  [(171.61, 184.62),
   (147.9, 172.9, 'Dark Counter Attack'),
   (165.96, 205.96, 'Dark Offense'),
   (171.61, 184.62, 'Dark Shot')],
  [(242.29, 255.34), (242.29, 255.34, 'Dark Shot')],
  [(270.71, 283.73),
   (261.94, 286.95, 'Dark Counter Attack'),
   (270.71, 283.73, 'Dark Shot')],
  [(326.38, 339.39), (326.38, 339.39, 'Dark Shot')],
  [(368.62, 383.64),
   (357.91, 382.91, 'Dark Counter Attack'),
   (368.62, 381.63, 'Dark Goal'),
   (370.63, 383.64, 'Dark Shot')],
  [(540.1216666667, 553.1416666667),
   (540.1216666667, 553.1416666667, 'Dark Shot')],
  [(604.1416666667, 617.1516666667),
   (604.1416666667, 617.1516666667, 'Dark Shot'),
   (614.2816666667, 639.2916666667, 'Dark Counter Attack')],
  [(686.5816666667, 699.5916666667),
   (681.5916666667, 706.5916666667, 'Dark Counter Attack'),
   (686.5816666667, 699.5916666667

In [ ]:
# get that clip and save it.
vid_save_loc = '{}/{}'.format(rootdir,game_metadata.game_file.values[game_id])
save_path = '/Volumes/scet/CalWWP-2_2017-18/temp_clips/'

g_code = 0
event_id = 7

inversion = [2,1,0]; codec = 'rawvideo'
inversion = [0,1,2]; codec = 'libx264'

# for avi rawvideo, inversion: [2,1,0]
def invert_image(image):
    return image[:,:,inversion]

#modifiedClip = my_clip.fl_image( invert_green_blue )

for (k,id) in enumerate(ids):
    print(k)
    t_start = float(events[id]['start'])
    t_end = float(events[id]['end'])
    clip = VideoFileClip(vid_save_loc).subclip(t_start, t_end)
    # clip name convention GAME_CODE +  event_id + clip number
    clip_name = '{}_{}_{}.avi'.format(g_code, event_id, k)
    clip_path = os.path.join(save_path, clip_name)
    # t_end - t_start
    clip.set_duration(t_end-t_start).fl_image(invert_image).write_videofile(clip_path,codec=codec)
    
    del clip
